In [20]:
# Import Splinter and BeautifulSoup
from splinter import Browser
from bs4 import BeautifulSoup as soup
from webdriver_manager.chrome import ChromeDriverManager
import pandas as pd

In [21]:
executable_path = {'executable_path': ChromeDriverManager().install()}
browser = Browser('chrome', **executable_path, headless=False)     #  headless set to FALSE because we were running in Jupyter notebook and want to see scraping



====== WebDriver manager ======
Current google-chrome version is 92.0.4515
Get LATEST driver version for 92.0.4515
Driver [C:\Users\reach\.wdm\drivers\chromedriver\win32\92.0.4515.107\chromedriver.exe] found in cache


In [22]:
# Visit the mars nasa news site
url = 'https://redplanetscience.com'
browser.visit(url)
# Optional delay for loading the page
browser.is_element_present_by_css('div.list_text', wait_time=1)

True

One is that we're searching for elements with a specific combination of tag (div) and attribute (list_text). As an example, ul.item_list would be found in HTML as <ul class="item_list">.

Secondly, we're also telling our browser to wait one second before searching for components. The optional delay is useful because sometimes dynamic pages take a little while to load, especially if they are image-heavy.

In [23]:
#  set up HTML parser 
html = browser.html
news_soup = soup(html, 'html.parser')
slide_elem = news_soup.select_one('div.list_text')              #slide_elem  is avariable which will look for div tag   This is our parent element

In [24]:
#  looking for a <div /> with a class of “content_title.”
slide_elem.find('div', class_='content_title')

<div class="content_title">Mars Scientists Investigate Ancient Life in Australia</div>

In [25]:
#  created a new variable for the title, added the get_text() method, and  searching within the parent element for the title.
#   Earlier, we identified the parent element and created a variable to hold it. With this new code, we’re searching within that element for the title. 
#   We’re also stripping the additional HTML attributes and tags with the use of .get_text().
#   need just text ad no extra elements of HTML 
# Use the parent element to find the first `a` tag and save it as `news_title`
news_title = slide_elem.find('div', class_='content_title').get_text()
news_title

'Mars Scientists Investigate Ancient Life in Australia'

In [26]:
#  searching for the summary instead of the title, so we’ll need to use the unique class associated with the summary

# Use the parent element to find the paragraph text
news_p = slide_elem.find('div', class_='article_teaser_body').get_text()
news_p

"Teams with NASA's Mars 2020 and ESA's ExoMars practiced hunting for fossilized microbial life in the Australian Outback in preparation for their Red Planet missions. "

# Scrape Mars Data: Featured Image

In [27]:
#  set up our browser for this new url 

# Visit URL
url = 'https://spaceimages-mars.com'
browser.visit(url)

In [28]:
#  # Find and click the full image button
full_image_elem = browser.find_by_tag('button')[1]
full_image_elem.click()

In [29]:
# Parse the resulting html with soup
html = browser.html
img_soup = soup(html, 'html.parser')

In [30]:
# Find the relative image url
img_url_rel = img_soup.find('img', class_='fancybox-image').get('src')
img_url_rel

'image/featured/mars1.jpg'

In [31]:
# Use the base URL to create an absolute URL
img_url = f'https://spaceimages-mars.com/{img_url_rel}'
img_url

'https://spaceimages-mars.com/image/featured/mars1.jpg'

# Scrape Mars Data: Mars Facts

In [32]:
# HTML are basically made up of many smaller containers. 
# The main container is the <table /> tag. Inside the table is <tbody />, which is the body of the table—the headers, columns, and rows

#  Instead of scraping each row, or the data in each <td />, we're going to scrape the entire table with Pandas' .read_html() function.

In [33]:
#  Pandas function read_html() specifically searches for and returns a list of tables found in the HTML. 
# By specifying an index of 0, we're telling Pandas to pull only the first table it encounters, or the first item in the list. 
#  Then, it turns the table into a DataFrame
df = pd.read_html('https://galaxyfacts-mars.com')[0]    #    creating a new DataFrame from the HTML table. The Pandas function read_html()
df.columns=['description', 'Mars', 'Earth']                      #   assign columns to the new DataFrame for additional clarity
df.set_index('description', inplace=True)                     #   using the .set_index() function, we're turning the Description column into the DataFrame's index. inplace=True means that the updated index will remain in place, without having to reassign the DataFrame to a new variable
df

,Mars,Earth
description,,
Mars - Earth Comparison,Mars,Earth
Diameter:,"6,779 km","12,742 km"
Mass:,6.39 × 10^23 kg,5.97 × 10^24 kg
Moons:,2,1
Distance from Sun:,"227,943,824 km","149,598,262 km"
Length of Year:,687 Earth days,365.24 days
Temperature:,-87 to -5 °C,-88 to 58°C


In [34]:
#  Convert this dataframe table to HTML 
df.to_html()

'<table border="1" class="dataframe">\n  <thead>\n    <tr style="text-align: right;">\n      <th></th>\n      <th>Mars</th>\n      <th>Earth</th>\n    </tr>\n    <tr>\n      <th>description</th>\n      <th></th>\n      <th></th>\n    </tr>\n  </thead>\n  <tbody>\n    <tr>\n      <th>Mars - Earth Comparison</th>\n      <td>Mars</td>\n      <td>Earth</td>\n    </tr>\n    <tr>\n      <th>Diameter:</th>\n      <td>6,779 km</td>\n      <td>12,742 km</td>\n    </tr>\n    <tr>\n      <th>Mass:</th>\n      <td>6.39 × 10^23 kg</td>\n      <td>5.97 × 10^24 kg</td>\n    </tr>\n    <tr>\n      <th>Moons:</th>\n      <td>2</td>\n      <td>1</td>\n    </tr>\n    <tr>\n      <th>Distance from Sun:</th>\n      <td>227,943,824 km</td>\n      <td>149,598,262 km</td>\n    </tr>\n    <tr>\n      <th>Length of Year:</th>\n      <td>687 Earth days</td>\n      <td>365.24 days</td>\n    </tr>\n    <tr>\n      <th>Temperature:</th>\n      <td>-87 to -5 °C</td>\n      <td>-88 to 58°C</td>\n    </tr>\n  </tbody>

In [35]:
browser.quit()     #  end the browser session you created 

# Export to Python